In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from PIL import Image
from utils import cargar_y_preprocesar_imagen, predecir_imagen
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Ejercicio

Se va a crear un modelo con redes neuronales convolucionales para clasificar imágenes en perros y gatos

Necesitamos estandarizar las imágenes para evitar errores y desvíos durante el entrenamiento del modelo.

Vamos convertirlas a RGB y a 64 x 64. También normalizamos todas las imágenes a 255 pixeles. Los píxeles de una imagen en color, en el formato estándar, tienen valores que van de 0 a 255 para cada canal de color (rojo, verde y azul). Esto se debe a que cada canal se representa con un byte, que puede tomar 256 valores diferentes.

Al dividir cada píxel por 255 escala estos valores al rango [0, 1]. Esto significa que después de la normalización, todos los valores de los píxeles estarán entre 0 y 1.

Creamos un archivo utils.py que contenga esta función para que podamos llamarla al entrenar y predecir.

## Cargado de archivos

In [2]:
directorio_actual = os.getcwd()
print("Directorio actual de trabajo:", directorio_actual)
print("Contenido del directorio actual:", os.listdir(directorio_actual))

Directorio actual de trabajo: /Users/n.c.rodriguez/nico/curso_AI/Redes Neuronales/dogs_vs_cats
Contenido del directorio actual: ['.DS_Store', '__pycache__', 'kagglecatsanddogs_3367a', 'predict', 'utils.py', 'modelo_cats_vs_dogs.keras', '.ipynb_checkpoints', 'Clasificador_Imagenes_dogs_vs_cats.ipynb']


In [3]:
# Se define la ruta de las carpetas de perros y gatos

carpeta_gatos = "kagglecatsanddogs_3367a/PetImages/Cat"
carpeta_perros = "kagglecatsanddogs_3367a/PetImages/Dog"

ruta_gatos = os.path.join(directorio_actual, carpeta_gatos)
ruta_perros = os.path.join(directorio_actual, carpeta_perros)

In [4]:
# Cargar y procesar imágenes
imagenes = []
etiquetas = []

In [5]:
# Cargar gatos
for archivo in os.listdir(ruta_gatos):
    ruta_completa = os.path.join(ruta_gatos, archivo)
    #lista de extensiones permitidas
    extensiones_permitidas = ['.jpg', '.jpeg', '.png', '.bmp']

    _, extension = os.path.splitext(archivo)

    #comprobar que la extensión es una de las permitidas
    if (extension.lower() in extensiones_permitidas):
        try:
            img = cargar_y_preprocesar_imagen(ruta_completa)
            imagenes.append(img)
            etiquetas.append(0) # 0 para gatos
        except Exception as e:
            print(f"No se pudo cargar la imagen {archivo}: {str(e)}")
            continue
    else:
        continue

In [6]:
# Cargar perros
for archivo in os.listdir(ruta_perros):
    ruta_completa = os.path.join(ruta_perros, archivo)
    #lista de extensiones permitidas
    extensiones_permitidas = ['.jpg', '.jpeg', '.png', '.bmp']

    _, extension = os.path.splitext(archivo)

    #comprobar que la extensión es una de las permitidas
    if (extension.lower() in extensiones_permitidas):
        try:
            img = cargar_y_preprocesar_imagen(ruta_completa)
            imagenes.append(img)
            etiquetas.append(1) # 1 para perros
        except Exception as e:
            print(f"No se pudo cargar la imagen {archivo}: {str(e)}")
            continue
    else:
        continue

/Users/n.c.rodriguez/anaconda3/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:866: UserWarning: Truncated File Read
  warnings.warn(str(msg))


## Construcción del modelo

In [7]:
# Convertir las listas imagenes y etiquetas a arrays de numpy
imagenes = np.array(imagenes)
etiquetas = np.array(etiquetas)

In [8]:
# Se divide la muestra en train y test
X_train, X_test, y_train, y_test = train_test_split(imagenes, etiquetas,test_size=0.2, random_state=42)

In [9]:
# Construcción del modelo
modelo = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [10]:
# Compilación del modelo
modelo.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
# Entrenamiento del modelo
modelo.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
624/624 [==============================] - 13s 21ms/step - loss: 0.6136 - accuracy: 0.6598 - val_loss: 0.5293 - val_accuracy: 0.7388
Epoch 2/10
624/624 [==============================] - 13s 21ms/step - loss: 0.4989 - accuracy: 0.7601 - val_loss: 0.4665 - val_accuracy: 0.7776
Epoch 3/10
624/624 [==============================] - 15s 23ms/step - loss: 0.4463 - accuracy: 0.7923 - val_loss: 0.4770 - val_accuracy: 0.7710
Epoch 4/10
624/624 [==============================] - 14s 23ms/step - loss: 0.3984 - accuracy: 0.8187 - val_loss: 0.4248 - val_accuracy: 0.8033
Epoch 5/10
624/624 [==============================] - 14s 23ms/step - loss: 0.3582 - accuracy: 0.8390 - val_loss: 0.4211 - val_accuracy: 0.8077
Epoch 6/10
624/624 [==============================] - 14s 23ms/step - loss: 0.3079 - accuracy: 0.8663 - val_loss: 0.4197 - val_accuracy: 0.8185
Epoch 7/10
624/624 [==============================] - 14s 22ms/step - loss: 0.2592 - accuracy: 0.8914 - val_loss: 0.4669 - val_accuracy:

In [12]:
# Evaluar el modelo
evaluacion = modelo.evaluate(X_test, y_test)
print(f"Test Accuracy: {evaluacion[1] * 100}%")

156/156 [==============================] - 1s 6ms/step - loss: 0.5958 - accuracy: 0.8119
Test Accuracy: 81.18990659713745%


El 80.37% de las imágenes en el conjunto de prueba fueron clasificadas correctamente por el modelo

In [13]:
modelo.save('modelo_cats_vs_dogs.keras') # Guarda el modelo en un archivo .keras

## Predicción usando el modelo

In [14]:
from keras.models import load_model

In [15]:
modelo = load_model('modelo_cats_vs_dogs.keras')

In [17]:
predecir_imagen('predict/perro1.jpg')
predecir_imagen('predict/perro2.jpg')
predecir_imagen('predict/perro3.jpg')
predecir_imagen('predict/gato.jpg')
predecir_imagen('predict/gato2.jpg')
predecir_imagen('predict/unknown.jpg')

1/1 [==============================] - 0s 60ms/step
Es un perro
1/1 [==============================] - 0s 25ms/step
Es un gato
1/1 [==============================] - 0s 19ms/step
Es un perro
1/1 [==============================] - 0s 17ms/step
Es un perro
1/1 [==============================] - 0s 20ms/step
Es un gato
1/1 [==============================] - 0s 19ms/step
Es un perro
